In [1]:
from pathlib import Path

import trimesh
import numpy as np
import h5py


In [2]:
def load_acronym_mesh(
    dataset_path: Path, object_class: str, grasp_uuid: str
) -> trimesh.Trimesh:
    """Loads an object from the Acronym dataset.
    Centers the mesh at the origin and scales the mesh according to the scale stored in the dataset.

    Args:
        dataset_path (Path): Path to the dataset
        object_class (str): Object class
        grasp_uuid (str): UUID of the grasp
    """
    
    assert object_class in [
        p.name for p in (dataset_path / "grasps").iterdir()
    ], f"Object class {object_class} not found in dataset path {dataset_path}/grasps"

    grasp_file_path = list(
        (dataset_path / "grasps" / object_class).glob(
            f"{object_class}_{grasp_uuid}*.h5"
        )
    )

    assert len(grasp_file_path) != 0, f"Grasp file not found: {grasp_file_path}"
    assert len(grasp_file_path) < 2, f"Multiple grasp files found: {grasp_file_path}"
    grasp_file_path = grasp_file_path[0]

    grasp_data = h5py.File(grasp_file_path, "r")
    mesh_scale = grasp_data["object"]["scale"][()]
    mesh_file_path = (
        dataset_path
        / "meshes"
        / grasp_data["object"]["file"][()].decode("utf-8")[len("meshes") + 1 :]
    )

    mesh = trimesh.load_mesh(mesh_file_path)
    if type(mesh) == trimesh.scene.scene.Scene:
        mesh = trimesh.util.concatenate(mesh.dump())
    mesh = mesh.apply_translation(-mesh.centroid)
    mesh = mesh.apply_scale(mesh_scale)

    return mesh

In [4]:
ACRONYM_PATH = Path("/home/data")

for object_class_path in (ACRONYM_PATH / "grasps").iterdir():
    for grasp_file_path in object_class_path.iterdir():
        grasp_data = h5py.File(grasp_file_path, "r")

        mesh_scale = grasp_data["object"]["scale"][()]
        mesh_file_path = (
            ACRONYM_PATH
            / "meshes"
            / grasp_data["object"]["file"][()].decode("utf-8")[len("meshes") + 1 :]
        )

mesh = trimesh.load_mesh(mesh_file_path)
if type(mesh) == trimesh.scene.scene.Scene:
    mesh = trimesh.util.concatenate(mesh.dump())
mesh = mesh.apply_translation(-mesh.centroid)
mesh = mesh.apply_scale(mesh_scale)

AssertionError: Multiple grasp files found: [PosixPath('/home/data/grasps/PianoKeyboard/PianoKeyboard_423d14787208578580ff80bf789cd_0.004542245094319145.h5'), PosixPath('/home/data/grasps/PianoKeyboard/PianoKeyboard_423d14787208578580ff80bf789cd_0.003650831028787507.h5')]